# Improts

In [2]:
## Imports
import numpy as np
import pandas as pd
from matplotlib.patches import Patch
import itertools
import pickle
from matplotlib import pyplot as plt
import datetime
import scipy
from scipy import optimize
from scipy.sparse import csr_matrix
import sys
sys.path.append('../SEIR_full/')
sys.path.append('..')
import os
from SEIR_full.indices import *

#############################
# Generating interventions  #
#############################
## Must be run after cell parameters set to specific cell division

""" Eras explanation:
-first days of routine from Feb 21st - March 13th
-first days of no school from March 14th - March 16th
-without school and work from March 17th - March 25th
-100 meters constrain from March 26th - April 2nd
-Bnei Brak quaranrine from April 3rd
"""


### Setting parameters:
# market_pct of 10 means lockdown
market_pct = 75
no_risk = False
no_school = False
no_kid10 = True

### creating notations for intervention.
with (open('../Data/parameters/indices.pickle', 'rb')) as openfile:
	ind = pickle.load(openfile)

In [17]:
inter_name = ind.cell_name + '@' + str(market_pct)
if  market_pct != 10:
    inter_name += '_no_haredim1317_half_jerus'
    
### importing files for manipulation:
# full_mtx ordering
full_mtx_home = scipy.sparse.load_npz('../Data/base_contact_mtx/full_home.npz')

full_mtx_work = {
    'routine': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_routine.npz'),
    'no_school': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_school.npz'),
    'no_work': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_work.npz'),
    'no_100_meters': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_100_meters.npz'),
    'no_bb': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_bb.npz'),
}

full_mtx_leisure = {
    'routine': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_routine.npz'),
    'no_school': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_school.npz'),
    'no_work': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_work.npz'),
    'no_100_meters': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_100_meters.npz'),
    'no_bb': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_bb.npz'),
}
# stay_home ordering
stay_home_idx_school =  pd.read_csv('../Data/stay_home/no_school.csv',index_col=0)
stay_home_idx_school.index = stay_home_idx_school.index.astype(str)
stay_home_idx_work =  pd.read_csv('../Data/stay_home/no_work.csv',index_col=0)
stay_home_idx_work.index = stay_home_idx_work.index.astype(str)
stay_home_idx_routine =  pd.read_csv('../Data/stay_home/routine.csv',index_col=0)
stay_home_idx_routine.index = stay_home_idx_routine.index.astype(str)
stay_home_idx_no_100_meters =  pd.read_csv('../Data/stay_home/no_100_meters.csv',index_col=0)
stay_home_idx_no_100_meters.index = stay_home_idx_no_100_meters.index.astype(str)
stay_home_idx_no_bb =  pd.read_csv('../Data/stay_home/no_bb.csv',index_col=0)
stay_home_idx_no_bb.index = stay_home_idx_no_bb.index.astype(str)
# reordering and expanding vector for each period:
stay_home_idx_school = stay_home_idx_school['mean'].values
stay_home_idx_school[1] = stay_home_idx_school[0]

stay_home_idx_work = stay_home_idx_work['mean'].values
stay_home_idx_work[1] = stay_home_idx_work[0]

stay_home_idx_no_100_meters = stay_home_idx_no_100_meters['mean'].values
stay_home_idx_no_100_meters[1] = stay_home_idx_no_100_meters[0]

stay_home_idx_no_bb = stay_home_idx_no_bb['mean'].values
stay_home_idx_no_bb[1] = stay_home_idx_no_bb[0]
# expanding vectors:
stay_home_idx_school = expand_partial_array(mapping_dic=ind.region_ga_dict,array_to_expand=stay_home_idx_school,
                                              size=len(ind.GA))
stay_home_idx_work = expand_partial_array(mapping_dic=ind.region_ga_dict,array_to_expand=stay_home_idx_work,
                                              size=len(ind.GA))
stay_home_idx_no_100_meters = expand_partial_array(mapping_dic=ind.region_ga_dict,array_to_expand=stay_home_idx_no_100_meters,
                                              size=len(ind.GA))
stay_home_idx_no_bb = expand_partial_array(mapping_dic=ind.region_ga_dict,array_to_expand=stay_home_idx_no_bb,
                                              size=len(ind.GA))

### setting C_inter:
C_calibration = {}
d_tot = 500
home_inter = []
work_inter = []
leis_inter = []
home_no_inter = []
work_no_inter = []
leis_no_inter = []

### setting stay_home:
stay_idx_t_work_inter = []
stay_idx_t_not_work_inter = []
routine_vector_inter = []
stay_idx_t_non_inter = []
routine_vector_non_inter = []

# make baseline for intervention:
if market_pct == 10:
    work = full_mtx_work['no_100_meters']
    leisure = full_mtx_leisure['no_100_meters']

    stay_home_idx_work_inter_spec = stay_home_idx_no_100_meters.copy()
    stay_home_idx_not_work_inter_spec = \
        stay_home_idx_no_100_meters.copy()

elif market_pct == 30:
    work = full_mtx_work['no_work']
    leisure = full_mtx_leisure['no_work']

    stay_home_idx_work_inter_spec = stay_home_idx_work.copy()
    stay_home_idx_not_work_inter_spec = stay_home_idx_work.copy()

elif market_pct > 30:
    factor = (market_pct-30.0)/10.0
    work = full_mtx_work['no_work'] + \
           (full_mtx_work['routine'] - full_mtx_work['no_work']) * factor/7.0
    leisure = full_mtx_leisure['no_work'] + \
              (full_mtx_leisure['routine'] - full_mtx_leisure['no_work']) * factor/7.0

    stay_home_idx_work_inter_spec = stay_home_idx_work.copy() + \
                                    (np.ones_like(
                                        stay_home_idx_work) - stay_home_idx_work.copy()) \
                                    * factor / 7.0
    stay_home_idx_not_work_inter_spec = stay_home_idx_work.copy() + \
                                    (np.ones_like(
                                        stay_home_idx_work) - stay_home_idx_work.copy()) \
                                    * factor / 7.0

else:
    print('market_pct value is not define!')
    sys.exit()

# make inter for base
if market_pct != 10:
    for idx, (area, age) in ind.GA.items():
        if (age in ['10-19']) and (area in ['11_betshemesh', '51_bb', '11']):
#                 work[idx, :] = full_mtx_work['routine'][idx, :]
#                 work[:, idx] = full_mtx_work['routine'][:, idx]

#                 stay_home_idx_work_inter_spec[idx] = 1
            work[idx, :] = full_mtx_work['no_100_meters'][idx, :]
            work[:, idx] = full_mtx_work['no_100_meters'][:, idx]

            stay_home_idx_work_inter_spec[idx] = \
                stay_home_idx_no_100_meters[idx]

        if age in ['70+', '60-69']:
            if no_risk:
                work[idx, :] = full_mtx_work['no_100_meters'][idx, :]
                work[:, idx] = full_mtx_work['no_100_meters'][:, idx]
    
                leisure[idx, :] = full_mtx_leisure['no_100_meters'][idx, :]
                leisure[:, idx] = full_mtx_leisure['no_100_meters'][:, idx]
    
for i in range(d_tot):
    home_inter.append(full_mtx_home)
    work_inter.append(work)
    leis_inter.append(leisure)

# make baseline for intervention:
if market_pct == 10:
    work = full_mtx_work['no_100_meters']
    leisure = full_mtx_leisure['no_100_meters']

    stay_home_idx_work_inter_spec = stay_home_idx_no_100_meters.copy()
    stay_home_idx_not_work_inter_spec = \
        stay_home_idx_no_100_meters.copy()

elif market_pct == 30:
    work = full_mtx_work['no_work']
    leisure = full_mtx_leisure['no_work']

    stay_home_idx_work_inter_spec = stay_home_idx_work.copy()
    stay_home_idx_not_work_inter_spec = stay_home_idx_work.copy()

elif market_pct > 30:
    factor = (market_pct-30.0)/10.0
    work = full_mtx_work['no_work'] + \
           (full_mtx_work['routine'] - full_mtx_work['no_work']) * factor/7.0
    leisure = full_mtx_leisure['no_work'] + \
              (full_mtx_leisure['routine'] - full_mtx_leisure['no_work']) * factor/7.0

    stay_home_idx_work_inter_spec = stay_home_idx_work.copy() + \
                                    (np.ones_like(
                                        stay_home_idx_work) - stay_home_idx_work.copy()) \
                                    * factor / 7.0
    stay_home_idx_not_work_inter_spec = stay_home_idx_work.copy() + \
                                    (np.ones_like(
                                        stay_home_idx_work) - stay_home_idx_work.copy()) \
                                    * factor / 7.0

else:
    print('market_pct value is not define!')
    sys.exit()

# make inter for base
if market_pct != 10:
    for idx, (area, age) in ind.GA.items():
        if (age in ['10-19']) and (area in ['11_betshemesh', '51_bb', '11']):
            work[idx, :] = full_mtx_work['no_100_meters'][idx, :]
            work[:, idx] = full_mtx_work['no_100_meters'][:, idx]

            stay_home_idx_work_inter_spec[idx] = \
                stay_home_idx_no_100_meters[idx]
                
        if age in ['70+', '60-69']:
            if no_risk:
                work[idx, :] = full_mtx_work['no_100_meters'][idx, :]
                work[:, idx] = full_mtx_work['no_100_meters'][:, idx]
    
                leisure[idx, :] = full_mtx_leisure['no_100_meters'][idx, :]
                leisure[:, idx] = full_mtx_leisure['no_100_meters'][:, idx]
for i in range(d_tot):
    home_no_inter.append(full_mtx_home)
    work_no_inter.append(work)
    leis_no_inter.append(leisure)

C_calibration['home_inter'] = home_inter
C_calibration['work_inter'] = work_inter
C_calibration['leisure_inter'] = leis_inter
C_calibration['home_non'] = home_no_inter
C_calibration['work_non'] = work_no_inter
C_calibration['leisure_non'] = leis_no_inter

for i in range(d_tot):
    stay_idx_t_work_inter.append(stay_home_idx_work_inter_spec)
    stay_idx_t_not_work_inter.append(stay_home_idx_not_work_inter_spec)
    routine_vector_inter.append(np.ones_like(stay_home_idx_work))

for i in range(d_tot):
    stay_idx_t_non_inter.append(stay_home_idx_no_100_meters)
    routine_vector_non_inter.append(np.ones_like(stay_home_idx_work))

stay_idx_calibration = {
    'non_inter':{
        'work': stay_idx_t_non_inter,
        'not_work': stay_idx_t_non_inter
    },
    'inter': {
        'work': stay_idx_t_work_inter,
        'not_work': stay_idx_t_not_work_inter,
    }
}
routine_vector_calibration = {
    'non_inter':{
        'work': [1]*500,
        'not_work': [1]*500,
    },
    'inter': {
        'work': [1]*500,
        'not_work': [1]*500,
    }
}

### make transfer to inter:
transfer_pop = ind.region_risk_age_dict.copy()
for region, risk, age in transfer_pop.keys():
    transfer_pop[(region, risk, age)] = 1
    if no_risk:
        if (risk == 'High') or (age in ['70+', '60-69']):
            transfer_pop[(region, risk, age)] = 0.0
#     if (age in ['10-19']) and (region in ['11_betshemesh', '51_bb']):
#         transfer_pop[(region, risk, age)] = 5.0/10.0
#     if (age in ['10-19']) and (region in ['11']):
#         transfer_pop[(region, risk, age)] = 2.5/10.0

### Save

with open('../Data/interventions/C_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(C_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/interventions/stay_home_idx_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(stay_idx_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/interventions/routine_t_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(routine_vector_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Data/interventions/transfer_pop_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(transfer_pop, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Making Intervention 30% no old with kid 0-6 with half class

In [15]:
inter_name = '30_no_old_kid04'

## Calibration contact matrix

In [16]:
full_mtx_home = scipy.sparse.load_npz('../Data/base_contact_mtx/full_home.npz')

full_mtx_work = {
    'routine': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_routine.npz'),
    'no_school': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_school.npz'),
    'no_work': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_work.npz'),
    'no_100_meters': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_100_meters.npz'),
    'no_bb': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_bb.npz'),
}

full_mtx_leisure = {
    'routine': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_routine.npz'),
    'no_school': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_school.npz'),
    'no_work': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_work.npz'),
    'no_100_meters': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_100_meters.npz'),
    'no_bb': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_bb.npz'),
}

In [17]:
C_calibration = {}
d_tot = 500

In [18]:
# intervation are null groups
home_inter = []
work_inter = []
leis_inter = []

# intervention of all kids - but 0.5 the class sise - of routine
# no old in this
# factor = 1.0/2.0
factor = 1.0

work = full_mtx_work['no_work'].copy()
leisure = full_mtx_leisure['no_work'].copy()
# set children matrix
for group, idx in age_ga_dict.items():
#     work[]
    if group in ['0-4']:
        work[idx, :] = full_mtx_work['routine'][idx, :] * factor
        work[:, idx] = full_mtx_work['routine'][:, idx] * factor
        
for group, idx in age_ga_dict.items():
#     work[]
    if group in ['70+', '60-69']:
        work[idx, :] = full_mtx_work['no_100_meters'][idx, :]
        work[:, idx] = full_mtx_work['no_100_meters'][:, idx]
        
        leisure[idx, :] = full_mtx_leisure['no_100_meters'][idx, :]
        leisure[:, idx] = full_mtx_leisure['no_100_meters'][:, idx]

for i in range(d_tot):
    home_inter.append(full_mtx_home)
    work_inter.append(work)
    leis_inter.append(leisure)
print(work)
print(leisure)
    

  (0, 0)	207.53675003674692
  (0, 1)	5.355919591971113
  (0, 2)	1.4848748368190596
  (0, 3)	5.139156176789472
  (0, 4)	2.8970814784151537
  (0, 5)	28.019194236929188
  (0, 6)	1.964505187877623
  (0, 7)	0.29099808967036683
  (1, 0)	0.7422821982789035
  (1, 1)	16.157642042803523
  (1, 2)	22.672199460996897
  (1, 3)	21.99390504581813
  (1, 4)	27.517724855624245
  (1, 5)	24.125808051972793
  (1, 6)	12.938956372825576
  (1, 7)	0.8635934199063298
  (1, 8)	0.0005499095043546266
  (1, 9)	0.00623852021720959
  (1, 10)	0.16787657958287266
  (1, 11)	0.3062331054227428
  (1, 12)	0.2722896349429506
  (1, 13)	0.3725744289143241
  (1, 14)	0.206211815376139
  (1, 15)	0.22983750456407365
  (1, 16)	0.01172454240653509
  :	:
  (179, 152)	2.6866921771699817e-06
  (179, 154)	7.090449808883403e-07
  (179, 155)	5.922192595065881e-06
  (179, 156)	1.0165133976977068e-05
  (179, 157)	4.955896756852091e-06
  (179, 158)	0.001808295124455163
  (179, 159)	3.4619310082974145e-06
  (179, 160)	2.885711254469235e-06
  

In [19]:
# # first days of routine from Feb 21st - March 13th

# # first days of no school from March 14th - March 16th

# # without school and work from March 17th - March 25th

# # 100 meters constrain from March 26th - April 2nd
    
# # Bnei Brak quaranrine from April 3rd


In [20]:
# non Intervantion
home_no_inter = []
work_no_inter = []
leis_no_inter = []

# all days are 30% - no work no school but taking hard close for old people

work = full_mtx_work['no_work'].copy()
leisure = full_mtx_leisure['no_work'].copy()
for group, idx in age_ga_dict.items():
#     work[]
    if group in ['70+', '60-69']:
        work[idx, :] = full_mtx_work['no_100_meters'][idx, :]
        work[:, idx] = full_mtx_work['no_100_meters'][:, idx]
        
        leisure[idx, :] = full_mtx_leisure['no_100_meters'][idx, :]
        leisure[idx, :] = full_mtx_leisure['no_100_meters'][idx, :]

In [21]:
for i in range(d_tot):
    home_no_inter.append(full_mtx_home)
    work_no_inter.append(work)
    leis_no_inter.append(leisure)

In [22]:
C_calibration['home_inter'] = home_inter
C_calibration['work_inter'] = work_inter
C_calibration['leisure_inter'] = leis_inter
C_calibration['home_non'] = home_no_inter
C_calibration['work_non'] = work_no_inter
C_calibration['leisure_non'] = leis_no_inter

In [23]:
# Save
with open('../Data/interventions/C_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(C_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)

## stay_home_idx+not_routine

In [24]:
stay_home_idx_school =  pd.read_csv('../Data/stay_home/no_school.csv',index_col=0)
stay_home_idx_school.index = stay_home_idx_school.index.astype(str)
stay_home_idx_work =  pd.read_csv('../Data/stay_home/no_work.csv',index_col=0)
stay_home_idx_work.index = stay_home_idx_work.index.astype(str)
stay_home_idx_routine =  pd.read_csv('../Data/stay_home/routine.csv',index_col=0)
stay_home_idx_routine.index = stay_home_idx_routine.index.astype(str)
stay_home_idx_no_100_meters =  pd.read_csv('../Data/stay_home/no_100_meters.csv',index_col=0)
stay_home_idx_no_100_meters.index = stay_home_idx_no_100_meters.index.astype(str)
stay_home_idx_no_bb =  pd.read_csv('../Data/stay_home/no_bb.csv',index_col=0)
stay_home_idx_no_bb.index = stay_home_idx_no_bb.index.astype(str)

In [25]:
# reordering and expanding vector for each period:
stay_home_idx_routine = stay_home_idx_routine['mean'].values
stay_home_idx_routine[1] = stay_home_idx_routine[0] 

stay_home_idx_school = stay_home_idx_school['mean'].values
stay_home_idx_school[1] = stay_home_idx_school[0]

stay_home_idx_work = stay_home_idx_work['mean'].values
stay_home_idx_work[1] = stay_home_idx_work[0]

stay_home_idx_no_100_meters = stay_home_idx_no_100_meters['mean'].values
stay_home_idx_no_100_meters[1] = stay_home_idx_no_100_meters[0]

stay_home_idx_no_bb = stay_home_idx_no_bb['mean'].values
stay_home_idx_no_bb[1] = stay_home_idx_no_bb[0]

# expanding vectors:
stay_home_idx_routine = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_routine,
                                              size=len(GA))
stay_home_idx_school = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_school,
                                              size=len(GA))
stay_home_idx_work = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_work,
                                              size=len(GA))
stay_home_idx_no_100_meters = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_no_100_meters,
                                              size=len(GA))
stay_home_idx_no_bb = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_no_bb,
                                              size=len(GA))

In [26]:
# # preparing model objects:
# stay_idx_t = []
# routine_vector = []

# # first days of routine from Feb 21st - March 13th

# # first days of no school from March 14th - March 16th

# # without school and work from March 17th - March 25th

# # 100 meters constrain from March 26th - April 2nd

# # Bnei Brak quaranrine from April 3rd

In [27]:
# preparing model objects:
# inter - are kids group going back to kindergardens: 0-6
# should be like regular time time - stay_home_idx_work
stay_idx_t_inter = []
routine_vector_inter = []

routine_vector_inter_spec = np.ones_like(stay_home_idx_work)
for group, idx in age_ga_dict.items():
    if group in ['0-4']:
        routine_vector_inter_spec[idx] = 0.0


for i in range(d_tot):
    stay_idx_t_inter.append(stay_home_idx_work)
    routine_vector_inter.append(routine_vector_inter_spec)


# non_inter - are kids staying home: 7-70+
# should be like no shool and no work time - stay_home_idx_work
stay_idx_t_non_inter = []
routine_vector_non_inter = []

for i in range(d_tot):
    stay_idx_t_non_inter.append(stay_home_idx_no_100_meters)
    routine_vector_non_inter.append(1)

In [28]:
stay_idx_calibration = {
    'non_inter':{
        'work': stay_idx_t_non_inter,
        'not_work': stay_idx_t_non_inter
    }, 
    'inter': {
        'work': stay_idx_t_inter,
        'not_work': stay_idx_t_non_inter,
    }
}

routine_vector_calibration = {
    'non_inter':{
        'work': routine_vector_non_inter,
        'not_work': routine_vector_non_inter
    }, 
    'inter': {
        'work': routine_vector_inter,
        'not_work': routine_vector_non_inter,
    }
}

In [29]:
# save objects
with open('../Data/interventions/stay_home_idx_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(stay_idx_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../Data/interventions/routine_t_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(routine_vector_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)

## creating Transfer pop dict for intervention

In [31]:
transfer_pop = region_risk_age_dict.copy()
for region, risk, age in transfer_pop.keys():
        transfer_pop[(region, risk, age)] = 1.0
#     transfer_pop[(region, risk, age)] = 0.0
#     if (risk=='Low') and (age not in ['60-69', '70+']):
#         transfer_pop[(region, risk, age)] = 1.0
#     if (risk=='Low') and (age in ['0-4']):
#         transfer_pop[(region, risk, age)] = 1.0
#     if (risk=='Low') and (age in ['5-9']):
#         transfer_pop[(region, risk, age)] = 2.0/5.0

with open('../Data/interventions/transfer_pop_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(transfer_pop, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Making Intervention 30% no old with kid 0-6

In [76]:
inter_name = '30_no_old_no_intervention'

In [77]:
full_mtx_home = scipy.sparse.load_npz('../Data/base_contact_mtx/full_home.npz')

full_mtx_work = {
    'routine': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_routine.npz'),
    'no_school': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_school.npz'),
    'no_work': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_work.npz'),
    'no_100_meters': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_100_meters.npz'),
    'no_bb': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_bb.npz'),
}

full_mtx_leisure = {
    'routine': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_routine.npz'),
    'no_school': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_school.npz'),
    'no_work': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_work.npz'),
    'no_100_meters': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_100_meters.npz'),
    'no_bb': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_bb.npz'),
}

C_calibration = {}
d_tot = 500


# intervation are null groups
home_inter = []
work_inter = []
leis_inter = []

# intervention of all kids - but 0.5 the class sise - of routine
# no old in this
# factor = 1.0/2.0
factor = 1.0

work = full_mtx_work['no_work'].copy()
leisure = full_mtx_leisure['no_work'].copy()
# # set children matrix
# for group, idx in age_ga_dict.items():
# #     work[]
#     if group in ['0-4', '5-9']:
#         work[idx, :] = full_mtx_work['routine'][idx, :] * factor
#         work[:, idx] = full_mtx_work['routine'][:, idx] * factor
        
# for group, idx in age_ga_dict.items():
#     if group in ['70+', '60-69']:
#         work[idx, :] = full_mtx_work['no_100_meters'][idx, :]
#         work[:, idx] = full_mtx_work['no_100_meters'][:, idx]
        
#         leisure[idx, :] = full_mtx_leisure['no_100_meters'][idx, :]
#         leisure[:, idx] = full_mtx_leisure['no_100_meters'][:, idx]

for i in range(d_tot):
    home_inter.append(full_mtx_home)
    work_inter.append(work)
    leis_inter.append(leisure)
    
# non Intervantion
home_no_inter = []
work_no_inter = []
leis_no_inter = []

# all days are 30% - no work no school but taking hard close for old people

work = full_mtx_work['no_work'].copy()
leisure = full_mtx_leisure['no_work'].copy()
for group, idx in age_ga_dict.items():
    if group in ['70+', '60-69']:
        work[idx, :] = full_mtx_work['no_100_meters'][idx, :]
        work[:, idx] = full_mtx_work['no_100_meters'][:, idx]
        
        leisure[idx, :] = full_mtx_leisure['no_100_meters'][idx, :]
        leisure[idx, :] = full_mtx_leisure['no_100_meters'][idx, :]
        
        
for i in range(d_tot):
    home_no_inter.append(full_mtx_home)
    work_no_inter.append(work)
    leis_no_inter.append(leisure)
    
C_calibration['home_inter'] = home_inter
C_calibration['work_inter'] = work_inter
C_calibration['leisure_inter'] = leis_inter
C_calibration['home_non'] = home_no_inter
C_calibration['work_non'] = work_no_inter
C_calibration['leisure_non'] = leis_no_inter

# Save
with open('../Data/interventions/C_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(C_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [78]:
stay_home_idx_school =  pd.read_csv('../Data/stay_home/no_school.csv',index_col=0)
stay_home_idx_school.index = stay_home_idx_school.index.astype(str)
stay_home_idx_work =  pd.read_csv('../Data/stay_home/no_work.csv',index_col=0)
stay_home_idx_work.index = stay_home_idx_work.index.astype(str)
stay_home_idx_routine =  pd.read_csv('../Data/stay_home/routine.csv',index_col=0)
stay_home_idx_routine.index = stay_home_idx_routine.index.astype(str)
stay_home_idx_no_100_meters =  pd.read_csv('../Data/stay_home/no_100_meters.csv',index_col=0)
stay_home_idx_no_100_meters.index = stay_home_idx_no_100_meters.index.astype(str)
stay_home_idx_no_bb =  pd.read_csv('../Data/stay_home/no_bb.csv',index_col=0)
stay_home_idx_no_bb.index = stay_home_idx_no_bb.index.astype(str)

# reordering and expanding vector for each period:
stay_home_idx_routine = stay_home_idx_routine['mean'].values
stay_home_idx_routine[1] = stay_home_idx_routine[0] 

stay_home_idx_school = stay_home_idx_school['mean'].values
stay_home_idx_school[1] = stay_home_idx_school[0]

stay_home_idx_work = stay_home_idx_work['mean'].values
stay_home_idx_work[1] = stay_home_idx_work[0]

stay_home_idx_no_100_meters = stay_home_idx_no_100_meters['mean'].values
stay_home_idx_no_100_meters[1] = stay_home_idx_no_100_meters[0]

stay_home_idx_no_bb = stay_home_idx_no_bb['mean'].values
stay_home_idx_no_bb[1] = stay_home_idx_no_bb[0]

# expanding vectors:
stay_home_idx_routine = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_routine,
                                              size=len(GA))
stay_home_idx_school = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_school,
                                              size=len(GA))
stay_home_idx_work = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_work,
                                              size=len(GA))
stay_home_idx_no_100_meters = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_no_100_meters,
                                              size=len(GA))
stay_home_idx_no_bb = expand_partial_array(mapping_dic=region_ga_dict,array_to_expand=stay_home_idx_no_bb,
                                              size=len(GA))

# preparing model objects:
# inter - are kids group going back to kindergardens: 0-6
# should be like regular time time - stay_home_idx_work
stay_idx_t_inter = []
routine_vector_inter = []

routine_vector_inter_spec = np.ones_like(stay_home_idx_work)
# for group, idx in age_ga_dict.items():
#     if group in ['0-4', '5-9']:
#         routine_vector_inter_spec[idx] = 0.0


for i in range(d_tot):
    stay_idx_t_inter.append(stay_home_idx_work)
    routine_vector_inter.append(routine_vector_inter_spec)


# non_inter - are kids staying home: 7-70+
# should be like no shool and no work time - stay_home_idx_work
stay_idx_t_non_inter = []
routine_vector_non_inter = []

for i in range(d_tot):
    stay_idx_t_non_inter.append(stay_home_idx_work)
    routine_vector_non_inter.append(1)
    

stay_idx_calibration = {
    'non_inter':{
        'work': stay_idx_t_non_inter,
        'not_work': stay_idx_t_non_inter
    }, 
    'inter': {
        'work': stay_idx_t_inter,
        'not_work': stay_idx_t_non_inter,
    }
}

routine_vector_calibration = {
    'non_inter':{
        'work': routine_vector_non_inter,
        'not_work': routine_vector_non_inter
    }, 
    'inter': {
        'work': routine_vector_inter,
        'not_work': routine_vector_non_inter,
    }
}

# save objects
with open('../Data/interventions/stay_home_idx_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(stay_idx_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('../Data/interventions/routine_t_inter_' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(routine_vector_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [79]:
transfer_pop = region_risk_age_dict.copy()
for region, risk, age in transfer_pop.keys():
        transfer_pop[(region, risk, age)] = 0.0
#     transfer_pop[(region, risk, age)] = 0.0
#     if (risk=='Low') and (age not in ['60-69', '70+']):
#         transfer_pop[(region, risk, age)] = 1.0
#     if (risk=='Low') and (age in ['0-4']):
#         transfer_pop[(region, risk, age)] = 1.0
#     if (risk=='Low') and (age in ['5-9']):
#         transfer_pop[(region, risk, age)] = 2.0/5.0

with open('../Data/interventions/transfer_pop_inter' + inter_name + '.pickle', 'wb') as handle:
    pickle.dump(transfer_pop, handle, protocol=pickle.HIGHEST_PROTOCOL)